In [6]:
import json
import os
import numpy as np
import pandas as pd

test = pd.read_json('val.json', typ='frame', encoding='utf-8')
tag_song = pd.read_json('tag_song_pair.json', typ='frame', orient= 'table', encoding='utf-8')
song_meta = pd.read_json('song_meta.json', typ='frame', encoding='utf-8')

In [7]:
def give_tag_song(idx):
    id_ = test.loc[idx]['id'].astype(str)
    tags = test.loc[idx]['tags']
    songs = test.loc[idx]['songs']
    return id_, tags, songs

def take_date(idx):
    target_date = test.loc[idx]['updt_date']
    year = target_date[:4]
    month = target_date[5:7]
    day = target_date[8:10]
    date = year+month+day
    return int(date)

def valid_date(idx, id_):
    plylst_date = take_date(idx)
    song_date = song_meta[song_meta['id']==id_]['issue_date'].to_list()[0]
    if song_date < plylst_date: return True
    else: return False

def setTagNumLimit(list): return len(list)+10
def tagIsTen(list, limit): return len(list) == limit
def songIsHund(list): return len(list) == 100

def resultCase_tag_song(idx):
    id_, tags, songs = give_tag_song(idx)
    tagNumLimit = setTagNumLimit(tags)
    
    tag_count = [0 for _ in range(29160)]
    # 태그 빈도수
    
    for i in range(29160):
        for s in tag_song.loc[i]['song']:
            if s in songs:
                tag_count[i] += 1   

    song_count = {}
    # 곡 빈도수
    
    for s in songs:
        song_count[s] = 0
    for t in tags:
        for s in tag_song[tag_song['tag']==t]['song'].to_list()[0]:
            if s in songs:
                song_count[s] = song_count[s] + 1    
    # tag selection
    new_tags = []
    tag_count_copy = tag_count
    for i in range(tagNumLimit):
        target = tag_count_copy.index(max(tag_count_copy))
        if tag_count_copy[target] == 0: break
        if tag_song.loc[target]["tag"] not in new_tags: new_tags.append(tag_song.loc[target]['tag'])
        tag_count_copy[target] = 0   
    del tag_count_copy 
    if tagIsTen(new_tags, tagNumLimit) is False: print("[ERROR] Need More Tags")
    
    # song selection
    new_songs = []
    new_song_dic = {}
    left_num = 100
    for t in new_tags:
        for s in tag_song[tag_song['tag']==t]['song'].to_list()[0]:
            if s not in songs:
                if s not in new_song_dic:
                    new_song_dic[s] = 0
                else: new_song_dic[s] = new_song_dic[s]+1
    while(len(new_songs)<100):
        if len(new_song_dic.keys()) <= 0: break
        target = max(new_song_dic.keys(), key=(lambda k: new_song_dic[k]))
        if valid_date(idx, target) and target not in songs and target not in new_songs: new_songs.append(target)
        del new_song_dic[target]
    if songIsHund(new_songs) is False:
        new_song_dic = {}
        for i in range(29160):
            for s in tag_song.loc[i]['song']:
                if s not in songs:
                    if s not in new_song_dic:
                        new_song_dic[s] = 0
                    else: new_song_dic[s] = new_song_dic[s]+1
        left_num = 100 - len(new_songs)
        for i in range(left_num):
            if len(new_song_dic.keys()) <= 0: break
            target = max(new_song_dic.keys(), key=(lambda k: new_song_dic[k]))
            if valid_date(idx, target) and target not in songs and target not in new_songs: new_songs.append(target)
            del new_song_dic[target]
        if songIsHund(new_songs) is False: print("[ERROR] Need More Songs")
    print("<Playlist {}>".format(idx))
    print("- Before Filtering")
    print("-- [Tags #]: " + str(len(new_tags)) + ", [Songs #]: " + str(len(new_songs)))
    result_list = []
    result_list.append(id_)
    for t in new_tags:
        if t in tags: new_tags.remove(t)
    result_list.append(new_tags[:9])
    result_list.append(new_songs)
    print("- After Filtering")
    print("-- [Tags #]: " + str(len(new_tags)) + ", [Songs #]: " + str(len(new_songs)))
    return result_list

In [8]:
import time
start = time.time()
asd = resultCase_tag_song(20000)
print(time.time() - start)

<Playlist 20000>
- Before Filtering
-- [Tags #]: 12, [Songs #]: 100
- After Filtering
-- [Tags #]: 11, [Songs #]: 100
32.272006034851074


In [9]:
asd

['42728',
 ['일본', '일본노래', '일본음악', 'JPOP모음', '기분전환', '독특한', '명곡', '분위기', '감성', '드라이브'],
 [653791,
  657261,
  70506,
  700090,
  209127,
  606596,
  580527,
  288102,
  688062,
  1133,
  264103,
  624957,
  5274,
  365762,
  40175,
  204900,
  437232,
  569405,
  408698,
  82815,
  83007,
  246882,
  379541,
  151419,
  20855,
  612090,
  318530,
  582588,
  191226,
  422736,
  654874,
  327385,
  165381,
  624852,
  560378,
  397712,
  693110,
  465141,
  695025,
  73698,
  270324,
  336466,
  567139,
  206932,
  305501,
  239980,
  534974,
  469623,
  10961,
  469885,
  503210,
  372528,
  471691,
  45747,
  144983,
  669747,
  112983,
  81126,
  572970,
  278599,
  278832,
  17206,
  17310,
  411421,
  182116,
  509798,
  247655,
  673766,
  608666,
  576278,
  151518,
  249955,
  676491,
  579717,
  678303,
  548304,
  287141,
  681541,
  92727,
  224364,
  454218,
  126691,
  291638,
  291668,
  128655,
  128839,
  654034,
  64052,
  32906,
  426292,
  524757,
  482,
  131757,
  49

In [10]:
for i in asd[2]:
    print(song_meta.loc[i]["song_name"])

RPG
Sparkle (movie ver.)
Yukino Hana (박효신 `눈의 꽃` 원곡)
なんでもないや (movie ver.) / Nandemonaiya (movie ver.)
Orion (NHK TV 애니메이션『3월의 라이온』2쿨 엔딩곡)
スパ-クル (movie ver.) / Sparkle (movie ver.)
Maboroshino Inochi (환상의 생명)
RAIN
前前前世 (movie ver.) / Zenzenzense (movie ver.)
あの夏へ / Ano Natsue (어느 여름날)
Best Friend
Kawaranai Mono (변하지 않는 것) (`시간을 달리는 소녀` 삽입곡)
Stars (Feat. Mari Mizuno Of Paris Match)
會いたくて 會いたくて / Aitakute Aitakute (보고 싶어서 보고 싶어서)
Sexual Method
Some Like It Hot!! Samurai Heart (애니메이션 `은혼` 엔딩 테마곡)
Moonrise...Moonset (Feat. Chieko Kinbara)
君はロックを聽かない / Kimiwa Rock Wo Kikanai (너는 록을 듣지 않아)
Desperado
Dragon Night
Say Anything
淚色 / Namidairo (눈물의 색)
花鳥風月 / Kachou Huugetsu (화조풍월)
キセキ(Kiseki)
Summer Of Farewells -From Up On Poppy Hill- (Vocals By Aoi Teshima)
Merry Christmas Mr. Lawrence - FYI
TT
Endless Rain
ピ&#12540;スサイン / Peace Sign
Kataomoi
Houki Boshi (혜성) (Album Mix Ver.)
Wherever You Are
君がくれた夏 / Kimiga Kureta Natsu (네가 준 여름)
Every Heart -Minnano Kimochi- (Every Heart -모두의 마음) (이누야샤 4기 ED)